# 아이리스 모델 만들기

In [1]:
import pandas as pd 
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('../static/data/iris_train.csv')
df_train.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.3,1.7,0.5,0
1,6.1,3.0,4.9,1.8,2
2,5.1,3.4,1.5,0.2,0


In [3]:
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_train.shape, y_train.shape

((112, 4), (112,))

In [4]:
df_test = pd.read_csv('../static/data/iris_test.csv')
df_test.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.8,3.0,1.4,0.3,0
1,4.6,3.6,1.0,0.2,0
2,4.8,3.0,1.4,0.1,0


In [5]:
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values
X_test.shape, y_test.shape

((38, 4), (38,))

In [6]:
scaler = StandardScaler()
scaler.fit(df_train.iloc[:, :-1])

StandardScaler()

In [7]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
joblib.dump(scaler, '../static/model/iris_scaler.pkl')

['../static/model/iris_scaler.pkl']

### 1. 로지스틱 회귀

In [9]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [10]:
params={
    # 'C':[0.1, 1, 3, 5]
    'C':[1, 1.5, 2]
}

In [11]:
grid_cv =GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9739
최적 파라미터: {'C': 1}


In [12]:
best_lr = grid_cv.best_estimator_
pred = best_lr.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8947368421052632

In [13]:
joblib.dump(best_lr, '../static/model/iris_lr.pkl')

['../static/model/iris_lr.pkl']

### 2. SVM

In [14]:
sv_clf=SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [15]:
params={
    # 'C':[0.1, 1, 3, 5]
    'C':[1, 1.5, 2]
}

In [16]:
grid_cv =GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled,y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9826
최적 파라미터: {'C': 1.5}


In [17]:
best_sv = grid_cv.best_estimator_
pred = best_sv.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.9210526315789473

In [18]:
joblib.dump(best_sv, '../static/model/iris_sv.pkl')

['../static/model/iris_sv.pkl']

### 3. Random Forest

In [19]:
rf_clf=RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [20]:
params = {
    # 'max_depth': [1, 3, 5, 10],
    'max_depth': [2, 3, 4],
    'min_samples_split': [1, 2, 3, 4]
}

In [21]:
grid_cv =GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled,y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9739
최적 파라미터: {'max_depth': 4, 'min_samples_split': 3}


In [22]:
best_rf = grid_cv.best_estimator_
pred = best_rf.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8947368421052632

In [23]:
joblib.dump(best_rf, '../static/model/iris_rf.pkl')

['../static/model/iris_rf.pkl']

### Test

In [24]:
index = 10
test_data = (df_test.iloc[index, :-1].values).reshape(1,-1)
label = df_test.iloc[index, -1]
test_data, label

(array([[6.9, 3.1, 5.4, 2.1]]), 2)

In [25]:
new_scaler = joblib.load('../static/model/iris_scaler.pkl')

In [26]:
test_scaled = new_scaler.transform(test_data)

In [27]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)

In [28]:
label, pred_lr[0], pred_sv[0], pred_rf[0]

(2, 2, 2, 2)

In [29]:
df_test.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.8,3.0,1.4,0.3,0
1,4.6,3.6,1.0,0.2,0
2,4.8,3.0,1.4,0.1,0


In [30]:
a = dict(zip(df_test.columns[:-1], df_test.iloc[index, :-1]))
a

{'sepal length (cm)': 6.9,
 'sepal width (cm)': 3.1,
 'petal length (cm)': 5.4,
 'petal width (cm)': 2.1}